![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [128]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [129]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

ds_jobs_transformed.head()

# Start coding here. Use as many cells as you like!

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


Columns containing categories with only two factors must be stored as Booleans (bool).


In [130]:
#Data Set as a Data Frame
ds_jobs_transformed

# Check all columns or a subset to see if they have exactly 2 unique values
unique_counts = ds_jobs_transformed.nunique()

# Filter and print columns with exactly 2 unique values
columns_with_two_values = unique_counts[unique_counts == 2]

print(columns_with_two_values)

relevant_experience    2
job_change             2
dtype: int64


In [131]:
# Normalize values, map to Boolean, and fill NaNs in one line. This is for relevant_experience variable
ds_jobs_transformed['relevant_experience'] = (
    ds_jobs_transformed['relevant_experience']
    .astype(str).str.strip().str.lower()
    .map({'has relevant experience': True, 'no relevant experience': False})
    .fillna(False)
)

# Map float values to Boolean: 1.0 -> True, 0.0 -> False, and handle NaNs by filling with False
ds_jobs_transformed['job_change'] = (
    ds_jobs_transformed['job_change']
    .map({1.0: True, 0.0: False})
    .fillna(False)  # Fill NaNs with False or any default you prefer
)

# Display the first few rows to verify the update
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,True,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,True
1,29725,city_40,0.776,Male,False,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,False
2,11561,city_21,0.624,NaN,False,Full time course,Graduate,STEM,5,NaN,NaN,never,83,False
3,33241,city_115,0.789,NaN,False,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,True
4,666,city_162,0.767,Male,True,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,False


Columns containing integers only must be stored as 32-bit integers (int32).


In [132]:
# Columns containing int64: student_id ; training_hours -> int32

ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32') 
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['student_id'].astype('int32')

Columns containing floats must be stored as 16-bit floats (float16).

In [133]:
# Columns containing float64; city_development_ index  -> float16

ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')


Columns containing nominal categorical data must be stored as the category data type.


In [138]:
# Columns with nominal catergorical data; city, gender, enrolled_university, education_level, major_discipline, experience, company_size, company_type, last_new_job

# Convert data types using .astype() with a dictionary
ds_jobs_transformed = ds_jobs_transformed.astype({
    'city': 'category',
    'gender': 'category',
    'enrolled_university': 'category',
    'education_level': 'category',
    'major_discipline': 'category',
    'experience': 'category',
    'company_size': 'category',
    'company_type': 'category',
    'last_new_job': 'category',
})

ds_jobs_transformed.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.


The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.